# TX00DQ05-3001 Exercises 3

In [4]:
import numpy as np
import numpy.linalg as LA
import random

## Exercise 1: Sample behaviour of an MDP

Let's take (again) a look at Sutton & Barto example 4.1 gridworld. On each iteration start at every (non-terminating) state and sample actions in succeeding states by selecting them from uniform distribution (each action - up, down, left, right - is equally probable). Run the episode until terminal state is encountered. Collect statistics to calculate average number of steps needed before completion for each start state. Should this number match with something you have seen earlier in the exercises?

In [69]:

def calculateValueForCurrentState(value,stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)


def checkWithBoundary(max,r,c,oR,oC):
    #print("-nstate: ",r,",",c)
    #print(V)
    if 0 > r or max <= r or 0 > c or max <= c :
        #print(oR,",",oC," value:",V[oR,oC])
        return (oR,oC)
    else:
        #print(r,",",c," value:",V[r,c])
        return (r,c)

    
    
def Actions(val):
    if val == 0:
        return [1,0]
    elif val == 1:
        return [-1,0]
    elif val == 2:
        return [0,1]
    elif val == 3:
        return [0,-1]
    

def randomAction():
    return Actions(random.randint(0,3))

def makeAction(maxsize,r,c):
    a = randomAction()
    
    
    return checkWithBoundary(maxsize,r+a[0],c+a[1],r,c)

def checkIfInTerminatingPoint(pos,term):
    for t in term:
        if pos == t:
            return True
    return False
    
class Agent(object):
    steps = 0
    def __init__(self, startPos):
        self.startPos = startPos
        self.pos = startPos
    
def MC_pathlengths(maxiters,gridworld,terminating):
    
    lengths= gridworld
    agents =[]
    
    for x in range(lengths.shape[0]):
        for y in range(lengths.shape[1]):
            if not checkIfInTerminatingPoint((x,y),terminating):
                agents.append( Agent((x,y)) )
    print(len(agents))
    for i in range(1, maxiters):

        goals = []
        calc = 0
        
        while (len(agents)>0):
            for agent in agents:
                agent.pos = makeAction(lengths.shape[0],agent.pos[0],agent.pos[1])
                agent.steps += 1
                if (checkIfInTerminatingPoint(agent.pos,terminating)):
                    agents.remove(agent)
                    goals.append(agent)
                    break
                    
            
        for agent in goals:
            mean = ((i-1)/i) * lengths[agent.startPos] + agent.steps/i
            #print(((i-1)/i),"*",lengths[agent.startPos],"+",agent.steps/i,"=",mean)
            lengths[agent.startPos] = mean
            agent.steps = 0
            agent.pos = agent.startPos
            
        agents = goals
    return lengths       
    
    
def CreateGridWorld(gridsize,terminating = None,numberOfTerminating = None):
    

    if(terminating == None):
        if(numberOfTerminating == 0 or numberOfTerminating == None):
            numberOfTerminating = 1
        terminating = []
        for n in range(numberOfTerminating):
            terminating.append((random.randint(0,gridsize[0]-1), \
                        random.randint(0,gridsize[1]-1)))
        
        
    gridworld = np.zeros((gridsize[0], gridsize[1]))
    
    print(terminating)
    print(gridworld)
    return gridworld, terminating


gridworld,terminating = CreateGridWorld((8,8),None,4)
with np.printoptions(precision=2):
    print(MC_pathlengths(1000,gridworld,terminating))

[(6, 4), (6, 6), (6, 3), (7, 4)]
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
60
[[117.2  110.14 111.42 105.89 105.71 107.31 107.04 103.17]
 [112.95 112.75 105.8  103.39 103.16 102.22  96.86  99.77]
 [103.25 101.15 100.93  94.58  94.44  95.74  91.95  93.17]
 [ 97.74  89.62  85.72  84.57  80.78  75.93  80.05  78.72]
 [ 88.89  83.31  67.21  64.95  60.22  57.86  59.6   56.39]
 [ 74.65  69.64  52.37  43.39  37.38  36.21  36.99  38.85]
 [ 66.73  55.13  39.1    0.     0.    10.48   0.    18.89]
 [ 57.61  55.33  36.7   12.29   0.     9.88   9.29  16.74]]


In [59]:
gridSize = (4,4)
actions = [
    (0,-1), #up
    (0, 1), #down
    (-1, 0),#left
    (1, 0)  #right
]
terminationPoints = [(0,0), (3,3)]

def terminationPoint(pos):
    for t in terminationPoints:
        if pos == t:
            return True
    return False

def doAction(pos):
    a = actions[random.randint(0,3)]
    newPos = tuple(map(lambda x, y: x + y, pos, a))
    # Check boundaries
    if (newPos[0] >= gridSize[0]):
        newPos = (gridSize[0]-1, pos[1])
    if (newPos[0] < 0):
        newPos = (0, pos[1])
    if (newPos[1] >= gridSize[1]):
        newPos = (pos[0], gridSize[1]-1)
    if (newPos[1] < 0):
        newPos = (pos[0], 0)
    
    return newPos

class Agent(object):
    steps = 0
    def __init__(self, startPos):
        self.startPos = startPos
        self.pos = startPos

def MC_pathlengths(maxiters):
    lengths = np.zeros(gridSize)
    
    # Initialize agents
    agentList = []
    aIndex = 0
    for x in range(gridSize[0]):
        for y in range(gridSize[1]):
            if not terminationPoint((x,y)):
                agentList.append( Agent((x,y)) )
            aIndex += 1
    for i in range(1, maxiters):
        # List for agents who have reached termination point
        stoppedAgentList = []
        
        # Loop while there is still agents who have not
        # reached term point
        while (len(agentList) > 0):
            for agent in agentList:
                agent.pos = doAction(agent.pos)
                agent.steps += 1
                if (terminationPoint(agent.pos)):
                    agentList.remove(agent)
                    stoppedAgentList.append(agent)
        
        # Calculate new mean and reset agents
        for agent in stoppedAgentList:
            mean = ((i-1)/i) * lengths[agent.startPos] + agent.steps/i
            #print(((i-1)/i),"*",lengths[agent.startPos],"+",agent.steps/i,"=",mean)
            lengths[agent.startPos] = mean
            agent.steps = 0
            agent.pos = agent.startPos
            
        #Put back for next iteration
        agentList = stoppedAgentList
            
    return lengths

print(MC_pathlengths(10))

[[ 0.         14.66666667 32.11111111 31.66666667]
 [10.22222222  9.44444444 16.88888889 14.22222222]
 [11.22222222 35.77777778 20.         24.55555556]
 [29.44444444 22.33333333 28.88888889  0.        ]]


In [49]:
# YOUR CODE

def MC_pathlengths(maxiters):
    lengths = dict()
    # YOUR CODE

    return lengths


## Exercise 2: Monte Carlo state value function estimation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1 using first-visit or every-visit Monte Carlo policy evaluation (see for example page 92 of Sutton & Barto). Policy to be evaluated is the same as before; each action (up, down, left, right) is equally probable.  Action that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

Try out both exploring starts (see Sutton & Barto, p. 96) and fixed start points. Any difference?

Take a look at the value function you get when you run the algorithm multiple times (with fixed # of iterations). Any observations?

[[  0.   -14.89 -20.72 -22.48]
 [-14.33 -18.32 -20.87 -21.31]
 [-20.25 -20.02 -18.3  -14.76]
 [-23.01 -20.63 -14.43   0.  ]]


## Exercise 3*: Monte Carlo action value function estimation

Use the same idea as in exercise 2 to estimate q function.

*) - not mandatory

## Exercise 4*: Monte Carlo control

Compute the optimal policy for the 4x4 gridworld example. Start with random policy. Consider the epsilon adjustment schedule - can it in practise be 1/k, or is something more conservative better? Can you think of any other tricks to manage the noisiness of MC?

*) - not mandatory